In [2]:
import edgar
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bsoup
import numpy as np
from collections import defaultdict
from cleanMDA import extractTable, divide_chunks, pullMDA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import SGDRegressor

In [5]:
#!conda install -y -c conda-forge xgboost

In [3]:
df = pd.read_pickle("Cleaned_MDA.pkl")

In [4]:
from collections import defaultdict
def getXy(df):
    '''Input is the Cleaned_MDA df output is the dataframe used to run machine learning algorithms '''
    dic = defaultdict()
    for year in df:
        text = []
        for i in range(len(df)):
            text.extend(df[year][i][1])
        dic[year] = text

    df = pd.DataFrame.from_dict([dic])


    gdp = pd.read_csv('gdp_annual.csv')
    USA = np.array(gdp[gdp['Data Source'] == 'United States'])
    gdp_USA = pd.DataFrame(USA, columns=['Country Name', 'Country COde', 'Indicator Name', 'Indicator Code',
                                                     '1960','1961','1962','1963','1964','1965','1966','1967','1968','1969',
                                                     '1970','1971','1972','1973','1974','1975','1976','1977','1978','1979',
                                                     '1980','1981','1982','1983','1984','1985','1986','1987','1988','1989',
                                                     '1990','1991','1992','1993','1994','1995','1996','1997','1998','1999',
                                                     '2000','2001','2002','2003','2004','2005','2006','2007','2008','2009',
                                                     '2010','2011','2012','2013','2014','2015','2016','2017','2018'])
    gdp_USA = gdp_USA.drop(columns=['Country Name', 'Country COde', 'Indicator Name', 'Indicator Code','1960','1961','1962','1963','1964','1965','1966','1967','1968','1969',
                                                     '1970','1971','1972','1973','1974','1975','1976','1977','1978','1979',
                                                     '1980','1981','1982','1983','1984','1985','1986','1987','1988','1989',
                                                     '1990','1991','1992','1993','1994','1995','1996','1997','1998','1999',
                                                     '2000','2001','2002','2003','2004','2005','2006','2018'])

    y = gdp_USA.transpose()
    X = df.transpose()
    result = pd.concat([X,y],axis=1)
    dataset = pd.DataFrame([np.array(X[0]),np.array(y[0])],columns=['2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017'])
    df = dataset.transpose()
    df.columns=['X','y']
    df['X'] = df['X'].map(lambda x: ' '.join(x))
    return df

In [6]:
df = getXy(df)

In [7]:
df.head()

,X,y
2007,exhibit financial statement schedule financial...,1.77857
2008,exhibit financial statement schedule financial...,-0.291621
2009,exhibit financial statement schedule financial...,-2.77553
2010,exhibit financial statement schedule financial...,2.53192
2011,exhibit financial statement schedule financial...,1.60145


In [21]:
#X_train, X_val, y_train, y_val = train_test_split(df['X'],df['y'], test_size=.2, random_state= 2)

In [22]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=2)
kf = KFold(n_splits=5, random_state=2, shuffle=True)
X = np.array(df['X'])
y = np.array(df['y'])

In [23]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [24]:
vectorizer = TfidfVectorizer(sublinear_tf=True, norm='l2', ngram_range=(1, 2))

In [25]:
#vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))

In [26]:
train_X_vec = vectorizer.fit_transform(X_train).toarray()

In [27]:
test_X_vec = vectorizer.transform(X_test).toarray()

In [28]:
# model = SGDRegressor(loss='squared_loss', penalty='l2', random_state=2, max_iter=5)
# params = {'penalty':['none','l2','l1'],
#           'alpha':[1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 0.1]}
# gs = GridSearchCV(estimator=model,
#                   param_grid=params,
#                   scoring='neg_mean_squared_error',
#                   n_jobs=1,
#                   cv=5,
#                   verbose=3)

In [41]:
model = xgb.XGBRegressor()

In [42]:
model.fit(train_X_vec, y_train)

[13:15:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [31]:
#gs.fit(train_X_vec, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] alpha=0.0001, penalty=none ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


[CV]  alpha=0.0001, penalty=none, score=-1.265327258954311, total=   0.6s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-7.2287051489534395, total=   0.1s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-2.8544295001811486, total=   0.0s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-3.774022010859653, total=   0.1s
[CV] alpha=0.0001, penalty=none ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
/anaconda3/lib

[CV]  alpha=0.0001, penalty=none, score=-1.460915924536781, total=   0.1s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-1.2653279018576415, total=   0.1s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-7.228704678224487, total=   0.1s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-2.854432041867558, total=   0.0s
[CV] alpha=0.0001, penalty=l2 ........................................


...

[CV] .. alpha=0.1, penalty=l1, score=-3.111579539683504, total=   0.1s
[CV] alpha=0.1, penalty=l1 ...........................................
[CV] .. alpha=0.1, penalty=l1, score=-4.031675231666001, total=   0.1s
[CV] alpha=0.1, penalty=l1 ...........................................


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


[CV] .. alpha=0.1, penalty=l1, score=-1.603060917118443, total=   0.2s


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:   14.8s finished
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=5,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=2, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'penalty': ['none', 'l2', 'l1'], 'alpha': [0.0001, 0.0002, 0.0005, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=3)

In [32]:
#model = gs.best_estimator_

In [44]:
test_X_vec.shape, y_test.shape 

((2, 309440), (2,))

In [45]:
y_test

array([2.861587025, 2.273338548], dtype=object)

In [46]:
model.predict(test_X_vec)

array([1.5123392, 1.4865376], dtype=float32)

In [47]:
model.score(test_X_vec, y_test)

-13.09982465515481

In [38]:
# print(gs.best_params_)
# print(gs.best_score_)

{'alpha': 0.0001, 'penalty': 'none'}
-3.5228759736037656


Now let's try some Sentiment Analysis

In [59]:
df

,X,y
2007,exhibit financial statement schedule financial...,1.77857
2008,exhibit financial statement schedule financial...,-0.291621
2009,exhibit financial statement schedule financial...,-2.77553
2010,exhibit financial statement schedule financial...,2.53192
2011,exhibit financial statement schedule financial...,1.60145
2012,management discussion analysis financial condi...,2.22403
2013,management discussion analysis financial condi...,1.67733
2014,management discussion analysis financial condi...,2.56919
2015,management discussion analysis financial condi...,2.86159
2016,management discussion analysis financial condi...,1.48528


In [69]:
! pip install -U textblob
! python -m textblob.download_corpora

from textblob import TextBlob

    100% |████████████████████████████████| 645kB 6.4MB/s ta 0:00:01
[nltk_data] Downloading package brown to /Users/david/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /Users/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/david/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/david/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /Users/david/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/david/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [70]:
def text_analysis(text):
   for word in text.split(' '):
       blob = TextBlob(word)
   #blob.tags # type of part of speech
       sent = blob.sentiment
       if sent.polarity != float(0):
           print(word,sent)

text_analysis(df['X']['2007'])

frequently Sentiment(polarity=0.1, subjectivity=0.3)
firm Sentiment(polarity=-0.2, subjectivity=0.4)
long Sentiment(polarity=-0.05, subjectivity=0.4)
long Sentiment(polarity=-0.05, subjectivity=0.4)
average Sentiment(polarity=-0.15, subjectivity=0.39999999999999997)
frequently Sentiment(polarity=0.1, subjectivity=0.3)
average Sentiment(polarity=-0.15, subjectivity=0.39999999999999997)
natural Sentiment(polarity=0.1, subjectivity=0.4)
available Sentiment(polarity=0.4, subjectivity=0.4)
common Sentiment(polarity=-0.3, subjectivity=0.5)
common Sentiment(polarity=-0.3, subjectivity=0.5)
common Sentiment(polarity=-0.3, subjectivity=0.5)
average Sentiment(polarity=-0.15, subjectivity=0.39999999999999997)
dry Sentiment(polarity=-0.06666666666666665, subjectivity=0.6)
long Sentiment(polarity=-0.05, subjectivity=0.4)
fixed Sentiment(polarity=0.1, subjectivity=0.2)
common Sentiment(polarity=-0.3, subjectivity=0.5)
average Sentiment(polarity=-0.15, subjectivity=0.39999999999999997)
common Sentime

higher Sentiment(polarity=0.25, subjectivity=0.5)
decreased Sentiment(polarity=-0.4, subjectivity=0.7)
absence Sentiment(polarity=-0.0125, subjectivity=0.0)
higher Sentiment(polarity=0.25, subjectivity=0.5)
common Sentiment(polarity=-0.3, subjectivity=0.5)
long Sentiment(polarity=-0.05, subjectivity=0.4)
outstanding Sentiment(polarity=0.5, subjectivity=0.875)
foreign Sentiment(polarity=-0.125, subjectivity=0.125)
foreign Sentiment(polarity=-0.125, subjectivity=0.125)
common Sentiment(polarity=-0.3, subjectivity=0.5)
outstanding Sentiment(polarity=0.5, subjectivity=0.875)
outstanding Sentiment(polarity=0.5, subjectivity=0.875)
decreased Sentiment(polarity=-0.4, subjectivity=0.7)
higher Sentiment(polarity=0.25, subjectivity=0.5)
common Sentiment(polarity=-0.3, subjectivity=0.5)
long Sentiment(polarity=-0.05, subjectivity=0.4)
outstanding Sentiment(polarity=0.5, subjectivity=0.875)
foreign Sentiment(polarity=-0.125, subjectivity=0.125)
foreign Sentiment(polarity=-0.125, subjectivity=0.125

fairly Sentiment(polarity=0.7, subjectivity=0.9)
straightforward Sentiment(polarity=0.375, subjectivity=0.375)
generally Sentiment(polarity=0.05000000000000002, subjectivity=0.5)
contingent Sentiment(polarity=-0.1, subjectivity=0.6)
important Sentiment(polarity=0.4, subjectivity=1.0)
effective Sentiment(polarity=0.6, subjectivity=0.8)
crude Sentiment(polarity=-0.7, subjectivity=1.0)
natural Sentiment(polarity=0.1, subjectivity=0.4)
natural Sentiment(polarity=0.1, subjectivity=0.4)
reasonable Sentiment(polarity=0.2, subjectivity=0.6)
economic Sentiment(polarity=0.2, subjectivity=0.2)
reasonable Sentiment(polarity=0.2, subjectivity=0.6)
reasonable Sentiment(polarity=0.2, subjectivity=0.6)
detailed Sentiment(polarity=0.4, subjectivity=0.75)
long Sentiment(polarity=-0.05, subjectivity=0.4)
professional Sentiment(polarity=0.1, subjectivity=0.1)
significant Sentiment(polarity=0.375, subjectivity=0.875)
notably Sentiment(polarity=0.5, subjectivity=0.5)
significant Sentiment(polarity=0.375, su

subject Sentiment(polarity=-0.16666666666666666, subjectivity=0.3333333333333333)
crude Sentiment(polarity=-0.7, subjectivity=1.0)
general Sentiment(polarity=0.05000000000000002, subjectivity=0.5)
dry Sentiment(polarity=-0.06666666666666665, subjectivity=0.6)
common Sentiment(polarity=-0.3, subjectivity=0.5)
common Sentiment(polarity=-0.3, subjectivity=0.5)
crude Sentiment(polarity=-0.7, subjectivity=1.0)
effective Sentiment(polarity=0.6, subjectivity=0.8)
doubtful Sentiment(polarity=-0.8, subjectivity=0.9)
crude Sentiment(polarity=-0.7, subjectivity=1.0)
long Sentiment(polarity=-0.05, subjectivity=0.4)
long Sentiment(polarity=-0.05, subjectivity=0.4)
long Sentiment(polarity=-0.05, subjectivity=0.4)
common Sentiment(polarity=-0.3, subjectivity=0.5)
foreign Sentiment(polarity=-0.125, subjectivity=0.125)
common Sentiment(polarity=-0.3, subjectivity=0.5)
common Sentiment(polarity=-0.3, subjectivity=0.5)
common Sentiment(polarity=-0.3, subjectivity=0.5)
foreign Sentiment(polarity=-0.125, s

generally Sentiment(polarity=0.05000000000000002, subjectivity=0.5)
straight Sentiment(polarity=0.2, subjectivity=0.4)
fixed Sentiment(polarity=0.1, subjectivity=0.2)
generally Sentiment(polarity=0.05000000000000002, subjectivity=0.5)
straight Sentiment(polarity=0.2, subjectivity=0.4)
fair Sentiment(polarity=0.7, subjectivity=0.9)
typically Sentiment(polarity=-0.16666666666666666, subjectivity=0.5)
generally Sentiment(polarity=0.05000000000000002, subjectivity=0.5)
firm Sentiment(polarity=-0.2, subjectivity=0.4)
fair Sentiment(polarity=0.7, subjectivity=0.9)
legal Sentiment(polarity=0.2, subjectivity=0.2)
impossible Sentiment(polarity=-0.6666666666666666, subjectivity=1.0)
due Sentiment(polarity=-0.125, subjectivity=0.375)
foreign Sentiment(polarity=-0.125, subjectivity=0.125)
other Sentiment(polarity=-0.125, subjectivity=0.375)
economic Sentiment(polarity=0.2, subjectivity=0.2)
greater Sentiment(polarity=0.5, subjectivity=0.5)
greater Sentiment(polarity=0.5, subjectivity=0.5)
first Se

...

fair Sentiment(polarity=0.7, subjectivity=0.9)
right Sentiment(polarity=0.2857142857142857, subjectivity=0.5357142857142857)
due Sentiment(polarity=-0.125, subjectivity=0.375)
generally Sentiment(polarity=0.05000000000000002, subjectivity=0.5)
certain Sentiment(polarity=0.21428571428571427, subjectivity=0.5714285714285714)
sec Sentiment(polarity=-0.1, subjectivity=0.6)
certain Sentiment(polarity=0.21428571428571427, subjectivity=0.5714285714285714)
significant Sentiment(polarity=0.375, subjectivity=0.875)
significant Sentiment(polarity=0.375, subjectivity=0.875)
significant Sentiment(polarity=0.375, subjectivity=0.875)
outstanding Sentiment(polarity=0.5, subjectivity=0.875)
outstanding Sentiment(polarity=0.5, subjectivity=0.875)
economic Sentiment(polarity=0.2, subjectivity=0.2)
fair Sentiment(polarity=0.7, subjectivity=0.9)
certain Sentiment(polarity=0.21428571428571427, subjectivity=0.5714285714285714)
fair Sentiment(polarity=0.7, subjectivity=0.9)
fair Sentiment(polarity=0.7, subjec

many Sentiment(polarity=0.5, subjectivity=0.5)
unexpected Sentiment(polarity=0.1, subjectivity=1.0)
higher Sentiment(polarity=0.25, subjectivity=0.5)
successful Sentiment(polarity=0.75, subjectivity=0.95)
general Sentiment(polarity=0.05000000000000002, subjectivity=0.5)
generally Sentiment(polarity=0.05000000000000002, subjectivity=0.5)
significantly Sentiment(polarity=0.375, subjectivity=0.875)
greater Sentiment(polarity=0.5, subjectivity=0.5)
subject Sentiment(polarity=-0.16666666666666666, subjectivity=0.3333333333333333)
generally Sentiment(polarity=0.05000000000000002, subjectivity=0.5)
certain Sentiment(polarity=0.21428571428571427, subjectivity=0.5714285714285714)
real Sentiment(polarity=0.2, subjectivity=0.30000000000000004)
approximately Sentiment(polarity=-0.4, subjectivity=0.6)
primarily Sentiment(polarity=0.4, subjectivity=0.5)
directly Sentiment(polarity=0.1, subjectivity=0.4)
fair Sentiment(polarity=0.7, subjectivity=0.9)
unexpected Sentiment(polarity=0.1, subjectivity=1.

whole Sentiment(polarity=0.2, subjectivity=0.4)
unpaid Sentiment(polarity=0.2, subjectivity=0.4)
subject Sentiment(polarity=-0.16666666666666666, subjectivity=0.3333333333333333)
right Sentiment(polarity=0.2857142857142857, subjectivity=0.5357142857142857)
general Sentiment(polarity=0.05000000000000002, subjectivity=0.5)
right Sentiment(polarity=0.2857142857142857, subjectivity=0.5357142857142857)
right Sentiment(polarity=0.2857142857142857, subjectivity=0.5357142857142857)
single Sentiment(polarity=-0.07142857142857142, subjectivity=0.21428571428571427)
right Sentiment(polarity=0.2857142857142857, subjectivity=0.5357142857142857)
full Sentiment(polarity=0.35, subjectivity=0.55)
least Sentiment(polarity=-0.3, subjectivity=0.4)
outstanding Sentiment(polarity=0.5, subjectivity=0.875)
unpaid Sentiment(polarity=0.2, subjectivity=0.4)
fixed Sentiment(polarity=0.1, subjectivity=0.2)
outstanding Sentiment(polarity=0.5, subjectivity=0.875)
unpaid Sentiment(polarity=0.2, subjectivity=0.4)
outst

approximately Sentiment(polarity=-0.4, subjectivity=0.6)
direct Sentiment(polarity=0.1, subjectivity=0.4)
fair Sentiment(polarity=0.7, subjectivity=0.9)
previously Sentiment(polarity=-0.16666666666666666, subjectivity=0.16666666666666666)
certain Sentiment(polarity=0.21428571428571427, subjectivity=0.5714285714285714)
fair Sentiment(polarity=0.7, subjectivity=0.9)
fair Sentiment(polarity=0.7, subjectivity=0.9)
better Sentiment(polarity=0.5, subjectivity=0.5)
fair Sentiment(polarity=0.7, subjectivity=0.9)
economically Sentiment(polarity=0.3, subjectivity=0.9)
fair Sentiment(polarity=0.7, subjectivity=0.9)
primarily Sentiment(polarity=0.4, subjectivity=0.5)
economically Sentiment(polarity=0.3, subjectivity=0.9)
fair Sentiment(polarity=0.7, subjectivity=0.9)
available Sentiment(polarity=0.4, subjectivity=0.4)
fair Sentiment(polarity=0.7, subjectivity=0.9)
fair Sentiment(polarity=0.7, subjectivity=0.9)
outstanding Sentiment(polarity=0.5, subjectivity=0.875)
fair Sentiment(polarity=0.7, sub

other Sentiment(polarity=-0.125, subjectivity=0.375)
small Sentiment(polarity=-0.25, subjectivity=0.4)
consistent Sentiment(polarity=0.25, subjectivity=0.25)
consistent Sentiment(polarity=0.25, subjectivity=0.25)
available Sentiment(polarity=0.4, subjectivity=0.4)
common Sentiment(polarity=-0.3, subjectivity=0.5)
long Sentiment(polarity=-0.05, subjectivity=0.4)
long Sentiment(polarity=-0.05, subjectivity=0.4)
long Sentiment(polarity=-0.05, subjectivity=0.4)
common Sentiment(polarity=-0.3, subjectivity=0.5)
common Sentiment(polarity=-0.3, subjectivity=0.5)
highly Sentiment(polarity=0.16, subjectivity=0.5399999999999999)
certain Sentiment(polarity=0.21428571428571427, subjectivity=0.5714285714285714)
certain Sentiment(polarity=0.21428571428571427, subjectivity=0.5714285714285714)
appropriately Sentiment(polarity=0.5, subjectivity=0.5)
foreign Sentiment(polarity=-0.125, subjectivity=0.125)
long Sentiment(polarity=-0.05, subjectivity=0.4)
primarily Sentiment(polarity=0.4, subjectivity=0.5)

In [73]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
blob = TextBlob(df['X']['2007'], analyzer=NaiveBayesAnalyzer())
blob.sentiment

Sentiment(classification='pos', p_pos=1.0, p_neg=0.0)